In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
pip install tensorflow==2.8.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### List the model names

In [ ]:
import os
PATH = '/content/gdrive'
data_path = PATH + '/My Drive/Colab Notebooks/Final Model/Pruning Model/Formatted_Pruning_Model/Pruned Model/MobileNetV2'
data_dir_list = os.listdir(data_path)

print(data_dir_list)

['Convert to PB_MobileNetV2.ipynb', 'MobileNetV2_New_baseline.h5', 'model_for_pruning.h5', 'model_for_pruning.pb']


In [ ]:
from keras import backend as K
# This line must be executed before loading Keras model.
K.set_learning_phase(0)

/usr/local/lib/python3.7/dist-packages/keras/backend.py:450: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


### Load the model and print first and last layer

In [ ]:
from keras.models import load_model
model_path = data_path + '/MobileNetV2_New_baseline.h5'

model = load_model(model_path)
print(model.outputs)
print(model.inputs)

[<tf.Tensor 'dense_1_6/Softmax:0' shape=(?, 21) dtype=float32>]
[<tf.Tensor 'input_1_6:0' shape=(?, 224, 224, 3) dtype=float32>]


### Freeze the model

In [ ]:
from tensorflow.keras.models import load_model
from keras import backend as K
import tensorflow as tf
from tensorflow.python.framework.ops import disable_eager_execution
from tensorflow.python.framework.graph_util import convert_variables_to_constants

tf.compat.v1.disable_eager_execution()


from tensorflow.keras.models import Model, load_model

tf.compat.v1.global_variables_initializer()

disable_eager_execution()

def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.compat.v1.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.compat.v1.global_variables()]
        # Graph -> GraphDef ProtoBuf
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                      output_names, freeze_var_names)
        return frozen_graph


frozen_graph = freeze_session(K.get_session(),
                              output_names=[out.op.name for out in model.outputs])

### Save the frozen model

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# Save to ./model/tf_model.pb
tf.train.write_graph(frozen_graph, data_path, "MobileNetV2_New_baseline.pb", as_text=False)

'/content/gdrive/My Drive/Colab Notebooks/Final Model/Pruning Model/Formatted_Pruning_Model/Pruned Model/MobileNetV2/MobileNetV2_New_baseline.pb'